In [1]:
pip install xgboost yfinance pandas numpy scikit-learn joblib gradio

In [6]:
# COMPLETE WORKING VERSION - COPY THIS ENTIRE CODE
# Run in Google Colab

# Install packages (run once)
# !pip install xgboost yfinance gradio scikit-learn --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import joblib
import os
import gradio as gr
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Threading fix
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

# Nifty 50 Stocks
NIFTY_50 = [
    'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'ITC.NS', 'KOTAKBANK.NS', 'LT.NS', 'SBIN.NS',
    'BHARTIARTL.NS', 'ASIANPAINT.NS', 'HCLTECH.NS', 'MARUTI.NS', 'AXISBANK.NS',
    'BAJFINANCE.NS', 'WIPRO.NS', 'SUNPHARMA.NS', 'NESTLEIND.NS', 'ULTRACEMCO.NS',
    'TITAN.NS', 'TATASTEEL.NS', 'TECHM.NS', 'POWERGRID.NS', 'INDUSINDBK.NS',
    'M&M.NS', 'NTPC.NS', 'HDFCLIFE.NS', 'ONGC.NS', 'JSWSTEEL.NS',
    'BAJAJFINSV.NS', 'GRASIM.NS', 'CIPLA.NS', 'SBILIFE.NS', 'TATAMOTORS.NS',
    'COALINDIA.NS', 'ADANIPORTS.NS', 'BPCL.NS', 'DIVISLAB.NS', 'BRITANNIA.NS',
    'APOLLOHOSP.NS', 'HEROMOTOCO.NS', 'EICHERMOT.NS', 'UPL.NS', 'DRREDDY.NS',
    'HINDALCO.NS', 'BAJAJ-AUTO.NS', 'TATACONSUM.NS', 'ADANIENT.NS', 'SHREECEM.NS'
]

def get_data(ticker, period='2y'):
    """Fetch stock data safely"""
    try:
        df = yf.download(ticker, period=period, progress=False)
        if len(df) > 0:
            return df
    except:
        pass
    return None

def add_features(df):
    """Add technical indicators"""
    data = df.copy()

    # Simple Moving Averages
    data['MA5'] = data['Close'].rolling(window=5).mean()
    data['MA10'] = data['Close'].rolling(window=10).mean()
    data['MA20'] = data['Close'].rolling(window=20).mean()

    # Volatility
    data['Volatility'] = data['Close'].rolling(window=10).std()

    # Price Returns
    data['Returns'] = data['Close'].pct_change()
    data['Returns5'] = data['Close'].pct_change(periods=5)

    # Volume Change
    data['VolumeChange'] = data['Volume'].pct_change()

    # High-Low Spread
    data['HLSpread'] = (data['High'] - data['Low']) / data['Close']

    # RSI Calculation
    delta = data['Close'].diff()
    gain = delta.copy()
    loss = delta.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    loss = abs(loss)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / (avg_loss + 1e-10)  # Avoid division by zero
    data['RSI'] = 100 - (100 / (1 + rs))

    # Target variable
    data['NextClose'] = data['Close'].shift(-1)

    # Clean data
    data = data.dropna()

    # Replace any inf/nan values
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        data[col] = data[col].replace([np.inf, -np.inf], np.nan)
        data[col] = data[col].fillna(data[col].mean())

    return data

def train_model_for_stock(ticker):
    """Train XGBoost model"""
    try:
        print(f"Training {ticker}...")

        # Get data
        raw_data = get_data(ticker, period='2y')
        if raw_data is None:
            return f"❌ Could not fetch data for {ticker}"

        # Add features
        data = add_features(raw_data)

        if len(data) < 100:
            return f"❌ Not enough data for {ticker} (need 100+ days)"

        # Define features
        feature_cols = ['Open', 'High', 'Low', 'Close', 'Volume',
                       'MA5', 'MA10', 'MA20', 'Volatility',
                       'Returns', 'Returns5', 'VolumeChange',
                       'HLSpread', 'RSI']

        X = data[feature_cols].values
        y = data['NextClose'].values

        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Train-test split (80-20)
        split_idx = int(len(X_scaled) * 0.8)
        X_train = X_scaled[:split_idx]
        X_test = X_scaled[split_idx:]
        y_train = y[:split_idx]
        y_test = y[split_idx:]

        # Train XGBoost
        model = XGBRegressor(
            n_estimators=100,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=1,
            verbosity=0
        )

        model.fit(X_train, y_train)

        # Evaluate
        predictions = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        r2 = r2_score(y_test, predictions)

        # Save model
        model_file = f'/tmp/{ticker.replace(".", "_")}_xgb.pkl'
        joblib.dump({
            'model': model,
            'scaler': scaler,
            'features': feature_cols
        }, model_file)

        return f"✅ {ticker} trained!\nRMSE: ₹{rmse:.2f} | R²: {r2:.3f}"

    except Exception as e:
        return f"❌ Training error: {str(e)}"

def predict_next_price(ticker):
    """Predict next closing price"""
    try:
        model_file = f'/tmp/{ticker.replace(".", "_")}_xgb.pkl'

        # Train if model doesn't exist
        if not os.path.exists(model_file):
            result = train_model_for_stock(ticker)
            if "❌" in result:
                return result

        # Load model
        saved = joblib.load(model_file)
        model = saved['model']
        scaler = saved['scaler']
        features = saved['features']

        # Get recent data
        raw_data = get_data(ticker, period='3mo')
        if raw_data is None:
            return f"❌ Could not fetch recent data for {ticker}"

        # Add features
        data = add_features(raw_data)

        if len(data) == 0:
            return f"❌ Not enough recent data for {ticker}"

        # Get latest row
        latest_row = data[features].iloc[-1].values.reshape(1, -1)
        latest_scaled = scaler.transform(latest_row)

        # Predict
        prediction = model.predict(latest_scaled)[0]

        # Get current price
        current_price = float(data['Close'].iloc[-1])
        price_change = prediction - current_price
        pct_change = (price_change / current_price) * 100

        # Get indicators
        ma20 = float(data['MA20'].iloc[-1])
        rsi_value = float(data['RSI'].iloc[-1])

        # Format output
        direction = "📈 BULLISH" if price_change > 0 else "📉 BEARISH"

        result = f"""
🏢 **{ticker.replace('.NS', '')} Stock Prediction**

💰 **Current Price:** ₹{current_price:.2f}
🔮 **Predicted Next Close:** ₹{prediction:.2f}

{direction}
📊 **Expected Change:** ₹{price_change:.2f} ({pct_change:+.2f}%)

📈 **Technical Indicators:**
• Moving Avg (20): ₹{ma20:.2f}
• RSI (14): {rsi_value:.1f}

⚠️ **Disclaimer:** This is an educational ML project.
Not financial advice. Do your own research!
"""
        return result

    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return f"❌ Prediction error: {str(e)}"

# ============================================
# GRADIO INTERFACE
# ============================================

with gr.Blocks(theme=gr.themes.Soft(), title="Stock Predictor") as demo:

    gr.Markdown("""
    # 📈 Nifty 50 Stock Price Predictor (XGBoost)

    ### ⚠️ **Educational Project - Not Financial Advice**
    This ML model uses XGBoost with technical indicators to predict next-day closing prices.
    Always consult financial advisors before investing.

    ---
    """)

    with gr.Tab("🔮 Predict Stock Price"):
        gr.Markdown("### Select a Nifty 50 stock to predict its next closing price")

        stock_select = gr.Dropdown(
            choices=NIFTY_50,
            value="RELIANCE.NS",
            label="📊 Select Stock"
        )

        predict_button = gr.Button("🚀 Predict Next Close", variant="primary", size="lg")

        prediction_output = gr.Textbox(
            label="Prediction Result",
            lines=14,
            show_copy_button=True
        )

        predict_button.click(
            fn=predict_next_price,
            inputs=stock_select,
            outputs=prediction_output
        )

    with gr.Tab("🎯 Train Model"):
        gr.Markdown("### Manually train or retrain models for specific stocks")

        train_select = gr.Dropdown(
            choices=NIFTY_50,
            value="TCS.NS",
            label="📊 Select Stock"
        )

        train_button = gr.Button("🔧 Train Model", variant="secondary", size="lg")

        train_output = gr.Textbox(
            label="Training Status",
            lines=5
        )

        train_button.click(
            fn=train_model_for_stock,
            inputs=train_select,
            outputs=train_output
        )

    gr.Markdown("""
    ---
    ### 📚 Model Information:
    - **Algorithm:** XGBoost (Gradient Boosting)
    - **Features:** 14 technical indicators (MA, RSI, Volatility, Returns)
    - **Training Data:** 2 years of historical prices
    - **Prediction:** Next day's closing price

    Built with ❤️ By Nikhil
    """)

# Launch app
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://24469da31aeba5b277.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://24469da31aeba5b277.gradio.live
